In [105]:
import pandas as  pd
import pymongo
import requests
import json
from datetime import datetime
import time
import calendar

In [106]:
bike_data_csv = "data/metro-bike-share-trips-2018-q3.csv"

In [125]:
bike_data_df = pd.read_csv(bike_data_csv)

In [112]:
bike_data_df['weekday'] = bike_data_df.apply(lambda row: calendar.day_name[datetime.strptime(row.start_time, '%Y-%m-%d %H:%M:%S').weekday()] , axis=1)

In [136]:
bike_data_df['start_day'] = bike_data_df.apply(lambda row: datetime.strptime(row.start_time, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') , axis=1)

In [115]:
bike_data_df.head()                           

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,weekday
0,94851140,8,2018-07-01 00:04:00,2018-07-01 00:12:00,3058,34.035801,-118.233170,3082,34.046520,-118.237411,6279,30,One Way,Monthly Pass,Sunday
1,94851141,8,2018-07-01 00:04:00,2018-07-01 00:12:00,3058,34.035801,-118.233170,3082,34.046520,-118.237411,6518,365,One Way,Flex Pass,Sunday
2,94851138,15,2018-07-01 00:09:00,2018-07-01 00:24:00,4147,34.145248,-118.150070,4174,34.165291,-118.150970,4823,365,One Way,Monthly Pass,Sunday
3,94851137,7,2018-07-01 00:22:00,2018-07-01 00:29:00,4157,34.140999,-118.132088,4162,34.147499,-118.148010,6115,30,One Way,Monthly Pass,Sunday
4,94851136,35,2018-07-01 00:23:00,2018-07-01 00:58:00,3013,33.779819,-118.263023,3013,33.779819,-118.263023,12055,1,Round Trip,Walk-up,Sunday


In [ ]:
bike_data_df['start_time'] = pd.to_datetime(bike_data_df['start_time'], format='%Y-%m-%d %H:%M:%S')
bike_data_df['end_time'] = pd.to_datetime(bike_data_df['end_time'], format='%Y-%m-%d %H:%M:%S')  

In [137]:
#items = bike_data_df[:1000].to_json(orient='records', date_format='iso', date_unit='s')
items = bike_data_df[:1000].to_json(orient='records')

In [38]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [39]:
db = client.bike_data_db
collection = db.bike_trip

In [40]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'bike_data_db'), 'bike_trip')

In [138]:
items_db = json.loads(items)

In [139]:
collection.drop()

In [140]:
db.bike_trip.insert_many(items_db)

In [141]:
collection.count_documents({})

1000

In [142]:
collection.distinct("start_day")

['2018-07-01', '2018-07-02']

In [135]:
collection.find_one()

{'_id': ObjectId('5c0127058a95a6039c6eb870'),
 'trip_id': 94851140,
 'duration': 8,
 'start_time': '2018-07-01 00:04:00',
 'end_time': '2018-07-01 00:12:00',
 'start_station': 3058,
 'start_lat': 34.035801,
 'start_lon': -118.23317,
 'end_station': 3082,
 'end_lat': 34.04652,
 'end_lon': -118.237411,
 'bike_id': 6279,
 'plan_duration': 30,
 'trip_route_category': 'One Way',
 'passholder_type': 'Monthly Pass'}

In [95]:
oneWayDoc = collection.find({"trip_route_category": "One Way"})

In [96]:
oneWayDoc[0]

{'_id': ObjectId('5c01197c8a95a6039c6eb0a0'),
 'trip_id': 94851140,
 'duration': 8,
 'start_time': '2018-07-01T00:04:00Z',
 'end_time': '2018-07-01T00:12:00Z',
 'start_station': 3058,
 'start_lat': 34.035801,
 'start_lon': -118.23317,
 'end_station': 3082,
 'end_lat': 34.04652,
 'end_lon': -118.237411,
 'bike_id': 6279,
 'plan_duration': 30,
 'trip_route_category': 'One Way',
 'passholder_type': 'Monthly Pass'}

In [123]:
#temp = collection.find({"start_time": {'$gte': '2018-07-01T00:00:00Z', '$lt': '2018-07-31T23:59:59Z'}})
temp = collection.find({"start_time": {'$gte': '2018-07-01 00:00:00', '$lt': '2018-07-31 23:59:59'}})

In [124]:
temp[2]

{'_id': ObjectId('5c0120b98a95a6039c6eb48a'),
 'trip_id': 94851138,
 'duration': 15,
 'start_time': '2018-07-01 00:09:00',
 'end_time': '2018-07-01 00:24:00',
 'start_station': 4147,
 'start_lat': 34.145248,
 'start_lon': -118.15007,
 'end_station': 4174,
 'end_lat': 34.165291,
 'end_lon': -118.15097,
 'bike_id': 4823,
 'plan_duration': 365,
 'trip_route_category': 'One Way',
 'passholder_type': 'Monthly Pass',
 'weekday': 'Sunday'}

In [97]:
datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%S.%fZ")

'20181130T111006.791030Z'